# H20 automl: Porto Seguro (Classification) # 2

Based on: https://github.com/h2oai/h2o-tutorials/blob/master/h2o-world-2017/automl/Python/automl_binary_classification_product_backorders.ipynb

In [1]:
time_limit = 60*60  # 1 hour max (confirm with timer)
metric = "AUC"  # specify your evaluation metric
target = "target" # name of target or label variable (just happens to be called "target" for this dataset)

## Import

In [2]:
# Import packages

from h2o.automl import H2OAutoML

import h2o
import numpy as np
import pandas as pd

In [3]:
# Import data
train_df = pd.read_csv("porto_train.csv")
test_df = pd.read_csv("porto_test.csv")

In [4]:
train_df.shape

(476170, 60)

In [5]:
test_df.shape

(119042, 59)

In [6]:
# initialize h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 40 mins
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,"1 year, 4 months and 24 days !!!"
H2O_cluster_name:,H2O_from_python_stever7_kk0ro3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,24.74 Gb
H2O_cluster_total_cores:,64
H2O_cluster_allowed_cores:,64
H2O_cluster_status:,"locked, healthy"


## Tidy

In [7]:
train_df = train_df.replace(-1, np.nan)
test_df = test_df.replace(-1, np.nan)

## Transform

In [8]:
# Convert categorical variables to "category" data type (after converting to h20 dataframe - easier)
cat_vars = [col for col in train_df.columns if 'cat' in col]
cat_vars

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [9]:
train_df = h2o.H2OFrame(train_df)
test_df = h2o.H2OFrame(test_df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
for col in cat_vars:
    test_df[col] = test_df[col].asfactor()
    
cat_vars = cat_vars + ["target"]

for col in cat_vars:
    train_df[col] = train_df[col].asfactor()

In [11]:
predictors = train_df.columns
predictors.remove(target)  # remove "target"
predictors.remove("id")  # remove "id" (should not be a predictor)
predictors.remove("fold")  # remove "fold" (not be a predictor)

## Visualize

(skipped)

## Model

In [12]:
%%time

aml = H2OAutoML(max_runtime_secs=time_limit, seed=1, sort_metric=metric)

CPU times: user 4.35 ms, sys: 0 ns, total: 4.35 ms
Wall time: 5.83 ms


In [13]:
%%time

aml.train(x=predictors, y=target, training_frame=train_df)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

15:47:30.871: GBM_lr_annealing_selection_AutoML_3_20230711_144728 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

CPU times: user 24.1 s, sys: 2.31 s, total: 26.4 s
Wall time: 1h 3s
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_4_AutoML_3_20230711_144728

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.03300307594641308
RMSE: 0.1816674873124332
LogLoss: 0.13905512270805367
Null degrees of freedom: 9852
Residual degrees of freedom: 9818
Null deviance: 3043.629546252695
Residual deviance: 2740.220248084906
AIC: 2810.220248084906
AUC: 0.7791122707618905
AUCPR: 0.18480011349747508
Gini: 0.558224541523781

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.069905515065112: 


,,0,1,Error,Rate
0,0,9050.0,450.0,0.0474,(450.0/9500.0)
1,1,240.0,113.0,0.6799,(240.0/353.0)
2,Total,9290.0,563.0,0.07,(690.0/9853.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.069906,0.246725,120.0
1,max f2,0.050678,0.313338,183.0
2,max f0point5,0.105031,0.264106,55.0
3,max accuracy,0.144126,0.965290,19.0
4,max precision,0.254252,1.000000,0.0
5,max recall,0.015726,1.000000,376.0
6,max specificity,0.254252,1.000000,0.0
7,max absolute_mcc,0.069906,0.218396,120.0
8,max min_per_class_accuracy,0.039408,0.699717,235.0
9,max mean_per_class_accuracy,0.041812,0.704392,222.0



Gains/Lift Table: Avg response rate:  3.58 %, avg score:  3.62 %


...

,,0,1,Error,Rate
0,0,416038.0,42773.0,0.0932,(42773.0/458811.0)
1,1,13583.0,3776.0,0.7825,(13583.0/17359.0)
2,Total,429621.0,46549.0,0.1184,(56356.0/476170.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.060664,0.118170,185.0
1,max f2,0.039040,0.199698,264.0
2,max f0point5,0.082931,0.099452,133.0
3,max accuracy,0.453869,0.963553,4.0
4,max precision,0.453869,0.750000,4.0
5,max recall,0.010240,1.000000,397.0
6,max specificity,0.538322,0.999998,0.0
7,max absolute_mcc,0.045988,0.082385,234.0
8,max min_per_class_accuracy,0.035304,0.600188,281.0
9,max mean_per_class_accuracy,0.035304,0.600371,281.0



Gains/Lift Table: Avg response rate:  3.65 %, avg score:  3.65 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010001,0.110916,3.525327,3.525327,0.128517,0.139862,0.128517,0.139862,0.035255,0.035255,252.532660,252.532660,0.026210
1,2,0.020001,0.094182,2.580631,3.052979,0.094078,0.101476,0.111298,0.120669,0.025808,0.061063,158.063124,205.297892,0.042616
2,3,0.030002,0.084954,2.246532,2.784163,0.081898,0.089273,0.101498,0.110204,0.022467,0.083530,124.653166,178.416317,0.055553
3,4,0.040000,0.078556,2.218196,2.642694,0.080865,0.081592,0.096341,0.103052,0.022179,0.105709,121.819578,164.269361,0.068194
4,5,0.050001,0.073838,2.142846,2.542720,0.078118,0.076115,0.092696,0.097664,0.021430,0.127139,114.284558,154.271980,0.080056
5,6,0.100000,0.060023,1.881484,2.212109,0.068590,0.066038,0.080643,0.081852,0.094072,0.221211,88.148430,121.210899,0.125797
6,7,0.150001,0.052419,1.485077,1.969762,0.054139,0.055936,0.071809,0.073213,0.074255,0.295466,48.507740,96.976174,0.150969
7,8,0.200000,0.047335,1.435597,1.836223,0.052335,0.049750,0.066940,0.067347,0.071778,0.367245,43.559671,83.622328,0.173572
8,9,0.300000,0.040364,1.194193,1.622213,0.043535,0.043594,0.059139,0.059430,0.119419,0.486664,19.419321,62.221326,0.193726
9,10,0.400000,0.035508,1.052480,1.479780,0.038369,0.037811,0.053946,0.054025,0.105248,0.591912,5.247998,47.977994,0.199173


In [14]:
%%time

lb = aml.leaderboard

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 17.4 µs


In [15]:
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_4_AutoML_3_20230711_144728,0.640933,0.151854,0.0670064,0.437851,0.186389,0.0347408
StackedEnsemble_AllModels_3_AutoML_3_20230711_144728,0.640878,0.151857,0.0671004,0.432658,0.186388,0.0347405
StackedEnsemble_BestOfFamily_4_AutoML_3_20230711_144728,0.63921,0.151984,0.0659994,0.429729,0.186432,0.034757
StackedEnsemble_AllModels_2_AutoML_3_20230711_144728,0.637616,0.152095,0.065378,0.43629,0.186459,0.034767
XGBoost_grid_1_AutoML_3_20230711_144728_model_16,0.637397,0.152084,0.0658065,0.436697,0.186446,0.0347621
StackedEnsemble_BestOfFamily_3_AutoML_3_20230711_144728,0.637164,0.152114,0.0652831,0.434707,0.186464,0.0347687
StackedEnsemble_AllModels_1_AutoML_3_20230711_144728,0.63517,0.152296,0.063508,0.434205,0.186529,0.0347929
GBM_grid_1_AutoML_3_20230711_144728_model_14,0.635137,0.152344,0.0639354,0.43322,0.186528,0.0347928
StackedEnsemble_BestOfFamily_2_AutoML_3_20230711_144728,0.63405,0.152354,0.0632191,0.431442,0.186542,0.034798
GBM_grid_1_AutoML_3_20230711_144728_model_28,0.633905,0.152481,0.0633598,0.434667,0.186575,0.0348102


In [ ]:
aml.leader

In [16]:
perf = aml.leader.model_performance(test_df)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.0347192088509328
RMSE: 0.18633091222589127
LogLoss: 0.15195377020399842
Null degrees of freedom: 119041
Residual degrees of freedom: 119007
Null deviance: 37231.77510716302
Residual deviance: 36177.76142524877
AIC: 36247.76142524877
AUC: 0.6358750380803229
AUCPR: 0.0662048016915021
Gini: 0.2717500761606457

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.05768761875560815: 


,,0,1,Error,Rate
0,0,102316.0,12391.0,0.108,(12391.0/114707.0)
1,1,3280.0,1055.0,0.7566,(3280.0/4335.0)
2,Total,105596.0,13446.0,0.1316,(15671.0/119042.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.057688,0.118666,175.0
1,max f2,0.040960,0.199026,240.0
2,max f0point5,0.094816,0.101821,95.0
3,max accuracy,0.363351,0.963618,5.0
4,max precision,0.363351,0.833333,5.0
5,max recall,0.010787,1.000000,396.0
6,max specificity,0.465173,0.999991,0.0
7,max absolute_mcc,0.050789,0.081307,199.0
8,max min_per_class_accuracy,0.034919,0.593913,268.0
9,max mean_per_class_accuracy,0.039664,0.596753,245.0



Gains/Lift Table: Avg response rate:  3.64 %, avg score:  3.63 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010005,0.109282,3.504636,3.504636,0.127624,0.135833,0.127624,0.135833,0.035063,0.035063,250.463617,250.463617,0.026006
1,2,0.020001,0.093717,2.884524,3.194710,0.105042,0.100709,0.116338,0.118278,0.028835,0.063899,188.452405,219.471033,0.045556
2,3,0.030006,0.084398,2.098170,2.829095,0.076406,0.088622,0.103024,0.108390,0.020992,0.084890,109.817034,182.909468,0.056958
3,4,0.040003,0.078213,2.076857,2.641114,0.075630,0.081140,0.096178,0.101580,0.020761,0.105652,107.685732,164.111432,0.068130
4,5,0.050008,0.073635,1.798432,2.472521,0.065491,0.075827,0.090039,0.096428,0.017993,0.123645,79.843172,147.252118,0.076420
5,6,0.100007,0.059841,1.822407,2.147491,0.066364,0.065874,0.078202,0.081152,0.091119,0.214764,82.240663,114.749121,0.119094
6,7,0.150006,0.052126,1.647086,1.980699,0.059980,0.055690,0.072129,0.072665,0.082353,0.297116,64.708650,98.069898,0.152670
7,8,0.200005,0.047032,1.310287,1.813103,0.047715,0.049447,0.066025,0.066861,0.065513,0.362630,31.028730,81.310310,0.168771
8,9,0.300003,0.040164,1.231855,1.619359,0.044859,0.043361,0.058970,0.059028,0.123183,0.485813,23.185461,61.935902,0.192832
9,10,0.400002,0.035380,0.971181,1.457318,0.035366,0.037668,0.053069,0.053688,0.097116,0.582930,-2.881875,45.731799,0.189841


## Communicate

(skipped)